In [19]:
import os
import sys
import pandas as pd
import openpyxl

import time
from datetime import datetime
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# NAVER BLOG Crawling
- title/url 

In [20]:
# 검색어 입력
query = input("검색어 : ")
location = str(query)

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# print(startDate, endDate)


# Chrome driver 환경설정 및 실행
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
driver = webdriver.Chrome(options = options)
base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
driver.get(base_url)


# 총 검색 결과
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
search_number = soup.find(class_="search_number")
# print("총 블로그글 개수 : ", search_number.text)


# 연관 검색어 
sub_keywords_data = soup.select('div.area_keyword > div.list')
sub_keywords = []
for i in sub_keywords_data: 
    sub_keywords.append(i.get_text())
# print("연관검색어:",*sub_keywords)


# blog url/title 가져오기
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
end_page=total_num//7+1
# print(end_page)

crawling_blog_data = [] 

try:
    for page_num in range(1,end_page):
        search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(search_url)
        time.sleep(0.1)
        # print(f'{page_num}번째 페이지 접속')

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        # print('html 가져오기 완료')

        # 블로그 제목, url, 포스팅 날짜 가져오기
        posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
        # print('posts_data 가져오기 완료')
        posts_date_data = soup.select('div.info_post > div.writer_info')
        # print('posts_date_data 가져오기 완료')

        for post, post_date in zip(posts_data, posts_date_data):
            title = post.text.replace('\n','').strip()
            href = post.attrs['href']
            date = post_date.find('span','date').text.strip()# Corrected this line
            try : 
                date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
            except: 
                date_timestamp = date
            crawling_blog_data.append({"location": location, "title": title, "url": href, "posted_date": date_timestamp})
        # print("리스트 담기 완료")

except Exception as e:
            print(f"error : {page_num}페이지 - {e}")

# print("carwling data :",crawling_blog_data)

In [22]:
len(crawling_blog_data)

420

In [23]:
df = pd.DataFrame(crawling_blog_data)
df


,location,title,url,posted_date
0,회기역,회기역 카페 - 회기역 근처 가볍게 점심 먹기 괜찮은 [ 읍천리382 ] 내돈사먹 후기,https://blog.naver.com/hyoddiya/223449731139,2024-05-17
1,회기역,회기역 호텔 모우 경희대 한국외대 근처숙소,https://blog.naver.com/mukgojayo/223453331578,2024-05-21
2,회기역,"위아짐 회기역pt 전문 헬스장, 피티 초보라면 조보경트레이너!",https://blog.naver.com/orot-i/223454276784,2024-05-22
3,회기역,"[교린이QnA]회기역 대학생 치아교정, '심미선'으로 나만의 맞춤 교정을 찾으세요",https://blog.naver.com/wenly7/223449724947,2024-05-17
4,회기역,호텔 모우 회기역 호텔 가성비 좋은 비지니스호텔,https://blog.naver.com/jems80/223449271410,2024-05-17
...,...,...,...,...
415,회기역,[나도 이런 창업해야지] LUCKYMEAL_럭키밀 어플 내돈내산 추천 ♡,https://blog.naver.com/mean0e/223448905481,2024-05-16
416,회기역,회기 위험한맛집 : 경성고기꾼 내돈내산,https://blog.naver.com/laboratory_man/22344775...,2024-05-16
417,회기역,[123] 회기역 역명판 <2024년>,https://blog.naver.com/minju4158/223452893026,2024-05-20
418,회기역,회기역 빵맛집 베이커리카페 964커피&너만보여크루아상,https://blog.naver.com/whao33/223449041413,2024-05-16


In [24]:
df1 = df.query('posted_date.str.contains("전")')
df1['posted_date'] = endDate
# df1.sort_values('posted_date', axis=0, ascending=False, inplace=True) 
df1

C:\Users\20061\AppData\Local\Temp\ipykernel_35924\2370409416.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['posted_date'] = endDate


,location,title,url,posted_date
34,회기역,회기역 월주차 2번출구 인근 편리한곳,https://blog.naver.com/plqh9295/223455804000,2024-05-23
38,회기역,"회기동 월주차 _ 회기역 도보 5분거리의 지정석,단일 주차장",https://blog.naver.com/zoomansa1/223455635624,2024-05-23
39,회기역,"행복주택 예비입주자 청약 신청 총정리 (월세 6만원부터,초역세권, 가좌역/회기역)",https://blog.naver.com/newchorok/223455224162,2024-05-23
40,회기역,회기역 맛집 회기왕갈비탕 점심 혼밥하기 좋은 외대 맛집,https://blog.naver.com/kwakyoukyeong/223455354721,2024-05-23
48,회기역,회기역 근처 맛집 밥집 '큐큐면관' - 5년 거주민이 데려가준 곳,https://blog.naver.com/joppdbsk/223450702877,2024-05-23
62,회기역,회기역 고기집 경희대맛집 회식장소 추천 숯불닭갈비 특수부위 팔각도,https://blog.naver.com/radiant_moments/2234556...,2024-05-23
79,회기역,[서울 회기역] 삼복 한방 닭한마리 후기,https://blog.naver.com/sh9510/223455123725,2024-05-23
81,회기역,동대문구 회기역 삼육보건대학교 VIP식당 고급도시락배달,https://blog.naver.com/brotherdosirak_nw/22344...,2024-05-23
85,회기역,[회기역 맛집] 혼밥하기 좋은 다이어트 키토김밥 맛집 삼청당 경희대점,https://blog.naver.com/bookstore85/223455770282,2024-05-23
96,회기역,"경희의료원 간병인, 회기역 경희의료원병원 간병인, 동대문구 경희대학교병원 간병인...",https://blog.naver.com/hadacare_/223455837401,2024-05-23


In [27]:
df2 = df.query('not posted_date.str.contains("시간 전")')
df2.sort_values('posted_date', axis=0, ascending=False, inplace=True) 
df2

C:\Users\20061\AppData\Local\Temp\ipykernel_35924\182116724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2.sort_values('posted_date', axis=0, ascending=False, inplace=True)


,location,title,url,posted_date
251,회기역,이문아이파크자이가 내년입주에서 밀리나요? (by.Xistarkim),https://blog.naver.com/skks7980/223455983170,40분 전
213,회기역,"대전, 서울 하이거, BYD 운행노선현황",https://blog.naver.com/angel060711/223454408552,2024-05-22
189,회기역,성북구운전면허학원 비용 평일 주말 연습 시험 저렴한운전면허학원 가격,https://blog.naver.com/letter9736/223454809429,2024-05-22
358,회기역,낯선 곳에서 재미를 추구하다,https://blog.naver.com/jjng1219/223454235615,2024-05-22
327,회기역,"ITX 청춘 정차역, 가평역/남춘천역/춘천역 경춘선 시간표",https://blog.naver.com/tmdqja1756/223454495743,2024-05-22
...,...,...,...,...
336,회기역,동대문구 실용음악학원 동대문구 드럼학원 동대문구 보컬학원 동대문구 댄스학원...,https://blog.naver.com/fwinds/223448265584,2024-05-16
334,회기역,24년 4월 일기(부동산 임장/초보 체험단/ADSP 독학/프리미엄 막걸리),https://blog.naver.com/nongnogne/223444593219,2024-05-16
259,회기역,경희대카페 스터디하기좋은 대형카페 _ 8번가 8street,https://blog.naver.com/mydreameruza/223448746614,2024-05-16
261,회기역,동대문미용실 악성곱슬 볼륨매직 예스헤어 만족합니다,https://blog.naver.com/co1823/223447979677,2024-05-16


In [28]:
total_df = pd.concat([df1,df2], ignore_index=True)
total_df

,location,title,url,posted_date
0,회기역,회기역 월주차 2번출구 인근 편리한곳,https://blog.naver.com/plqh9295/223455804000,2024-05-23
1,회기역,"회기동 월주차 _ 회기역 도보 5분거리의 지정석,단일 주차장",https://blog.naver.com/zoomansa1/223455635624,2024-05-23
2,회기역,"행복주택 예비입주자 청약 신청 총정리 (월세 6만원부터,초역세권, 가좌역/회기역)",https://blog.naver.com/newchorok/223455224162,2024-05-23
3,회기역,회기역 맛집 회기왕갈비탕 점심 혼밥하기 좋은 외대 맛집,https://blog.naver.com/kwakyoukyeong/223455354721,2024-05-23
4,회기역,회기역 근처 맛집 밥집 '큐큐면관' - 5년 거주민이 데려가준 곳,https://blog.naver.com/joppdbsk/223450702877,2024-05-23
...,...,...,...,...
416,회기역,동대문구 실용음악학원 동대문구 드럼학원 동대문구 보컬학원 동대문구 댄스학원...,https://blog.naver.com/fwinds/223448265584,2024-05-16
417,회기역,24년 4월 일기(부동산 임장/초보 체험단/ADSP 독학/프리미엄 막걸리),https://blog.naver.com/nongnogne/223444593219,2024-05-16
418,회기역,경희대카페 스터디하기좋은 대형카페 _ 8번가 8street,https://blog.naver.com/mydreameruza/223448746614,2024-05-16
419,회기역,동대문미용실 악성곱슬 볼륨매직 예스헤어 만족합니다,https://blog.naver.com/co1823/223447979677,2024-05-16


In [17]:
file_name = f"naver_crawling_{query}_{startDate}-{endDate}.csv"
total_df.to_csv(file_name, index=False)
data = pd.read_csv(file_name)
data

,location,title,url,posted_date
0,회기역,회기역 월주차 2번출구 인근 편리한곳,https://blog.naver.com/plqh9295/223455804000,2024-05-23
1,회기역,"회기동 월주차 _ 회기역 도보 5분거리의 지정석,단일 주차장",https://blog.naver.com/zoomansa1/223455635624,2024-05-23
2,회기역,"행복주택 예비입주자 청약 신청 총정리 (월세 6만원부터,초역세권, 가좌역/회기역)",https://blog.naver.com/newchorok/223455224162,2024-05-23
3,회기역,회기역 맛집 회기왕갈비탕 점심 혼밥하기 좋은 외대 맛집,https://blog.naver.com/kwakyoukyeong/223455354721,2024-05-23
4,회기역,회기역 근처 맛집 밥집 '큐큐면관' - 5년 거주민이 데려가준 곳,https://blog.naver.com/joppdbsk/223450702877,2024-05-23
...,...,...,...,...
416,회기역,동대문 이문동헬스장 / 헬스파크 / 가격주차리뷰,https://blog.naver.com/goodgameggyo/223448856449,2024-05-16
417,회기역,동대문구 실용음악학원 동대문구 드럼학원 동대문구 보컬학원 동대문구 댄스학원...,https://blog.naver.com/fwinds/223448265584,2024-05-16
418,회기역,24년 4월 일기(부동산 임장/초보 체험단/ADSP 독학/프리미엄 막걸리),https://blog.naver.com/nongnogne/223444593219,2024-05-16
419,회기역,경희대카페 스터디하기좋은 대형카페 _ 8번가 8street,https://blog.naver.com/mydreameruza/223448746614,2024-05-16
